In [1]:
import pandas as pd
import numpy as np

In [2]:
df_products = pd.read_csv('PRODUCTS.csv')
df_user = pd.read_csv('USER.CSV')
df_transaction = pd.read_csv('TRANSACTION.csv')

# Let's start off with finding many missing values are in each column.

### Creating a function to find the missing value percentage of each column.

In [15]:
def null_rates(df, df_name):

    null_rate = round(df.isnull().mean()*100, 2)
    null_rate_sort = null_rate.sort_values(ascending=False)

    print(f"\n--- Percentage of missing value for the {df_name} table---")
    print("{:<15} {:<5}".format('Column Name', 'Missing Value%'))
    for col, null in null_rate_sort.items():
        print("{:<20} {:<5}%".format(col, null))
    print('------------------------------------------------------------')

### Null rate function applied to each table to determine the null rates of each column

In [19]:
null_rates(df_products, 'Products')
null_rates(df_user, 'User')
null_rates(df_transaction, 'Transaction')


--- Percentage of missing value for the Products table---
Column Name     Missing Value%
CATEGORY_4           92.02%
MANUFACTURER         26.78%
BRAND                26.78%
CATEGORY_3           7.16 %
BARCODE              0.48 %
CATEGORY_2           0.17 %
CATEGORY_1           0.01 %
------------------------------------------------------------

--- Percentage of missing value for the User table---
Column Name     Missing Value%
LANGUAGE             30.51%
GENDER               5.89 %
STATE                4.81 %
BIRTH_DATE           3.68 %
ID                   0.0  %
CREATED_DATE         0.0  %
------------------------------------------------------------

--- Percentage of missing value for the Transaction table---
Column Name     Missing Value%
BARCODE              11.52%
RECEIPT_ID           0.0  %
PURCHASE_DATE        0.0  %
SCAN_DATE            0.0  %
STORE_NAME           0.0  %
USER_ID              0.0  %
FINAL_QUANTITY       0.0  %
FINAL_SALE           0.0  %
---------------------

# Based on the results, we notice a discrepancy in the Products table where the barcode is missing for about 0.48% of orders. We also see that the other unique identifiers missing for 26.78% of the values in the brands and manufacturer column. We can take a closer look at that see if there is any correlation.

### Here we're looking at the top 10 offenders of missing barcodes and not find any correlation
### the Barcode column with this many missing values is still an issue.

In [25]:
print(f"\n{df_products.loc[df_products['BARCODE'].isna(), 'MANUFACTURER'].value_counts(dropna=False).nlargest(10)}\n")



MANUFACTURER
COTY                      269
PROCTER & GAMBLE          268
NaN                       247
GENERAL MILLS             245
PEPSICO                   210
UNILEVER                  209
JOHNSON & JOHNSON         156
THE HERSHEY COMPANY       151
CHURCH & DWIGHT           142
MONDELĒZ INTERNATIONAL    140
Name: count, dtype: int64



### Next we want to validate if the datatype matches with the Entity Relationship Model schema provided in the aws page.


In [30]:
# Function for validating dtypes
def validate_dtypes(df, df_name=None):
    if df_name is None:
        df_name = "DataFrame"
    print(f"\n === Datatypes for {df_name} table ===")
    print("{:<15} {:<5}".format("Column Name", "Datatype"))
    for col, dt in df.dtypes.items():
        # print("{:<20} {:<5}".format(col, dt))
        print(f"{col}: {dt}")
    print('=========================================')

### Validating the dtype of each field which is mostly returning object so we will need to validate most numeric/dt fields
### Barcode is already float but can be converted to int but issue can arise if there are any values in the decimal places

In [33]:
validate_dtypes(df_products, 'Products')
validate_dtypes(df_user, 'User')
validate_dtypes(df_transaction, 'Transaction')


 === Datatypes for Products table ===
Column Name     Datatype
CATEGORY_1: object
CATEGORY_2: object
CATEGORY_3: object
CATEGORY_4: object
MANUFACTURER: object
BRAND: object
BARCODE: float64

 === Datatypes for User table ===
Column Name     Datatype
ID: object
CREATED_DATE: object
BIRTH_DATE: object
STATE: object
LANGUAGE: object
GENDER: object

 === Datatypes for Transaction table ===
Column Name     Datatype
RECEIPT_ID: object
PURCHASE_DATE: object
SCAN_DATE: object
STORE_NAME: object
USER_ID: object
BARCODE: float64
FINAL_QUANTITY: object
FINAL_SALE: object


### Now we can try validating certain columns one by one.\
### We're starting off with date validation, so worked on 2 columns here: scan_date, purchase_date

In [44]:
df_transaction["SCAN_DATE"] = pd.to_datetime(df_transaction["SCAN_DATE"])
scan_date_range = df_transaction["SCAN_DATE"].agg(['min', 'max'])
print(f'\nDate range of Scan Date from Transaction table:\n {scan_date_range}')


Date range of Scan Date from Transaction table:
 min   2024-06-12 06:36:34.910000+00:00
max   2024-09-08 23:07:19.836000+00:00
Name: SCAN_DATE, dtype: datetime64[ns, UTC]


In [46]:
df_transaction["PURCHASE_DATE"] = pd.to_datetime(df_transaction["PURCHASE_DATE"])
purchase_date_range = df_transaction["PURCHASE_DATE"].agg(['min', 'max'])
print(f'\nDate range of Purchase Date from Transaction table:\n {purchase_date_range}')


Date range of Purchase Date from Transaction table:
 min   2024-06-12
max   2024-09-08
Name: PURCHASE_DATE, dtype: datetime64[ns]


# Here we can see that the scan_date and purchase date matches up.

### Next, we will validate the birth_date and created_date from the User table:


In [54]:
df_user["BIRTH_DATE"] = pd.to_datetime(df_user["BIRTH_DATE"])
birth_date_range = df_user["BIRTH_DATE"].agg(['min', 'max'])
print(f'\nDate range of Birthdays from User table:\n {birth_date_range}')


Date range of Birthdays from User table:
 min   1900-01-01 00:00:00+00:00
max   2022-04-03 07:00:00+00:00
Name: BIRTH_DATE, dtype: datetime64[ns, UTC]


In [57]:
df_user["CREATED_DATE"] = pd.to_datetime(df_user["CREATED_DATE"])
created_date_range = df_user["CREATED_DATE"].agg(['min', 'max'])
print(f'\nDate range of Created Date from User table:\n {created_date_range}')


Date range of Created Date from User table:
 min   2014-04-18 23:14:55+00:00
max   2024-09-11 17:59:15+00:00
Name: CREATED_DATE, dtype: datetime64[ns, UTC]


# Based on the max birth date and min birth date, I am not sure if there should be anyone 2 years old or 124 years old making a purchase.
# Also there is a user created on September 11th but made a purchase 3 days prior.
# This needs to be verified but can be explained in cases like when a user uses guest checkout but comes back with the same email/phone number to create an account later on.

### Finally, we can validate the fields with int or float datatype from the schema

In [61]:
try:
    df_transaction["BARCODE"] = df_transaction["BARCODE"].dropna().apply(int)
    print('successfully converted BARCODE field of Transaction table to INT')
except ValueError as e:
    print('unable to convert BARCODE field of Transaction table to INT:', e)

successfully converted BARCODE field of Transaction table to INT


In [65]:
try:
    df_products["BARCODE"] = df_products["BARCODE"].dropna().apply(int)
    print('successfully converted BARCODE field of Products table to INT')
except ValueError as e:
    print('unable to convert BARCODE field of Products table to INT:', e)
print(f"\n")

successfully converted BARCODE field of Products table to INT




### We successfully converted the float table to int but not sure if there were any values in the decimal that could have been affected

### Now we can try to validate the numeric fields of quantity and sale of the transaction table.

In [71]:
try:
    df_transaction["FINAL_QUANTITY"] = df_transaction["FINAL_QUANTITY"].dropna().apply(float)
    print('successfully converted Final_Quantity field of Transaction table to Float')
except ValueError as e:
    print('unable to convert Final_Quantity field of Transaction table to Float:', e)
print(f"\n")

unable to convert Final_Quantity field of Transaction table to Float: could not convert string to float: 'zero'




### The quantity table is unable to convert so we can do a value counts to check it out.


In [74]:
print(df_transaction["FINAL_QUANTITY"].value_counts(dropna=False))

FINAL_QUANTITY
1.00    35698
zero    12500
2.00     1285
3.00      184
4.00      139
        ...  
6.22        1
1.22        1
1.23        1
2.57        1
2.27        1
Name: count, Length: 87, dtype: int64


# This is returning all sorts of values but I am going to assume some can be parts of a measurement like weight.
# But the main glaring issue I see here is the "zero" value being present.
# Not sure how purchases with "zero" quantity works but it should at least be a numeric value.
# There should a units field in this table if there are quantity that are in the decimals.

In [77]:
try:
    df_transaction["FINAL_SALE"] = df_transaction["FINAL_SALE"].dropna().apply(float)
    print('successfully converted Final_Sale field of Transaction table to Float')
except ValueError as e:
    print('unable to convert Final_Sale field of Transaction table to Float:', e)
print(f"\n")

unable to convert Final_Sale field of Transaction table to Float: could not convert string to float: ' '




### unable to convert due to empty spaces so lets remove that then try again.

In [80]:
try:
    if df_transaction["FINAL_SALE"].astype(str).str.contains(r'[^0-9\.\-\s]').any():
        raise ValueError("Contains alphabet str")

    # Filling the empty spaces/na values with 0
    df_transaction["FINAL_SALE"] = df_transaction["FINAL_SALE"].fillna(0)
    df_transaction["FINAL_SALE"] = df_transaction["FINAL_SALE"].astype(str).str.strip()
    df_transaction["FINAL_SALE"] = df_transaction["FINAL_SALE"].replace('', '0')
    df_transaction["FINAL_SALE"] = df_transaction["FINAL_SALE"].astype(float)
    print('successfully converted Final_Sale field of Transaction table to Float')
except ValueError as e:
    print('unable to convert Final_Sale field of Transaction table to Float:', e)
print(f"\n")

successfully converted Final_Sale field of Transaction table to Float




### After removing all the na values and empty spaces, we can successfully convert price to numeric.